<a href="https://colab.research.google.com/github/Murikaboshi/file-converter/blob/main/file_converter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install ezdxf geopandas shapely fiona pyproj tqdm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 72.4 MB/s eta 0:00:00


In [ ]:

import math, os
from typing import Iterable, Optional, List, Tuple

import ezdxf
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon

def to_xy(pt) -> Tuple[float, float]:
    """Normalize (x,y[,z]) or Vec2/Vec3 into (x,y)."""
    try:
        return float(pt[0]), float(pt[1])
    except Exception:
        return float(pt.x), float(pt.y)

def arc_points(cx: float, cy: float, r: float, start_deg: float, end_deg: float, n: int = 64):
    """Approximate a circle/arc by n segments."""
    s = math.radians(start_deg); e = math.radians(end_deg)
    if e < s: e += 2 * math.pi
    ts = np.linspace(s, e, max(2, n))
    xs = cx + r * np.cos(ts); ys = cy + r * np.sin(ts)
    return list(zip(xs, ys))

def dxf_to_geodataframe(
    dxf_path: str,
    layer_filter: Optional[Iterable[str]] = None,   # e.g. ["Road","River"]
    arc_segments: int = 96,                          # smoothness for CIRCLE/ARC
    circle_as_polygon: bool = True,                  # True: Polygon, False: LineString
    close_polyline_to_polygon: bool = False,         # treat closed polylines as Polygon
    crs_input: Optional[str] = None,                 # e.g. "EPSG:5186" if known
    crs_output: Optional[str] = "EPSG:4326",         # reproject (if crs_input is set)
) -> gpd.GeoDataFrame:
    doc = ezdxf.readfile(dxf_path)
    msp = doc.modelspace()

    allow = {str(x).lower() for x in layer_filter} if layer_filter else None
    rows: List[dict] = []

    for e in msp:
        if not hasattr(e, "dxftype"):
            continue
        et = e.dxftype()
        layer = getattr(e.dxf, "layer", None)
        if allow and (str(layer).lower() not in allow):
            continue

        geom = None
        try:
            if et == "LINE":
                geom = LineString([to_xy(e.dxf.start), to_xy(e.dxf.end)])

            elif et == "LWPOLYLINE":
                try:
                    pts = [(p[0], p[1]) for p in e.get_points("xy")]
                except Exception:
                    pts = [(p[0], p[1]) for p in e.points()]
                closed = bool(e.closed)
                if closed and close_polyline_to_polygon and len(pts) >= 3:
                    geom = Polygon(pts)
                else:
                    geom = LineString(pts + ([pts[0]] if closed else []))

            elif et == "POLYLINE":
                pts = []
                closed = bool(getattr(e, "is_closed", False))
                for v in e.vertices:
                    pts.append(to_xy(v.dxf.location))
                if closed and close_polyline_to_polygon and len(pts) >= 3:
                    geom = Polygon(pts)
                else:
                    geom = LineString(pts + ([pts[0]] if (closed and len(pts) >= 2) else []))

            elif et == "CIRCLE":
                cx, cy = to_xy(e.dxf.center); r = float(e.dxf.radius)
                ring = arc_points(cx, cy, r, 0.0, 360.0, arc_segments + 1)
                geom = Polygon(ring) if circle_as_polygon else LineString(ring)

            elif et == "ARC":
                cx, cy = to_xy(e.dxf.center); r = float(e.dxf.radius)
                a1, a2 = float(e.dxf.start_angle), float(e.dxf.end_angle)
                geom = LineString(arc_points(cx, cy, r, a1, a2, arc_segments))

            elif et == "POINT":
                geom = Point(to_xy(e.dxf.location))

        except Exception:
            # skip malformed/unexpected entities
            pass

        if geom is not None:
            rows.append({
                "layer": layer,
                "entity": et,
                "geometry": geom
            })

    gdf = gpd.GeoDataFrame(rows, geometry="geometry", crs=crs_input)

    # Only transform if an input CRS is set
    if crs_output and gdf.crs is not None:
        gdf = gdf.to_crs(crs_output)

    return gdf



In [ ]:
from google.colab import files

print("Upload a DXF file")
uploaded = files.upload()
assert len(uploaded) == 1, "Please upload exactly one DXF."
dxf_name = next(iter(uploaded.keys()))
base = os.path.splitext(os.path.basename(dxf_name))[0]

gdf = dxf_to_geodataframe(
    dxf_name,
    layer_filter=None,            # 取り出せる→例：["Road","River"]
    arc_segments=96,
    circle_as_polygon=True,
    close_polyline_to_polygon=False,
    crs_input=None,               # set like "EPSG:5186" only if you KNOW the DXF CRS
    crs_output="EPSG:4326",       # keep if you want WGS84 lon/lat output
)

print("features:", len(gdf))
display(gdf.head(3))

out_path = f"{base}.geojson"
gdf.to_file(out_path, driver="GeoJSON")
print("saved:", out_path)
files.download(out_path)

Upload a DXF file


Saving sample_ok.dxf to sample_ok.dxf
features: 5


,layer,entity,geometry
0,Layer1,LINE,"LINESTRING (0 0, 100 0)"
1,Layer1,CIRCLE,"POLYGON ((75 50, 74.94647 51.63508, 74.78612 5..."
2,Layer2,ARC,"LINESTRING (30 0, 29.9836 0.9919, 29.93441 1.9..."


saved: sample_ok.geojson


/usr/local/lib/python3.11/dist-packages/pyogrio/geopandas.py:710: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>